In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
import requests
from datetime import datetime

In [2]:
#Loading phishing sites file 
data0 = pd.read_csv("./data/verified_online.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7487021,https://attmailgfedrternbqwjhkmbfgpktygfghtdm....,http://www.phishtank.com/phish_detail.php?phis...,2022-04-12T17:01:01+00:00,yes,2022-04-12T17:12:32+00:00,yes,Other
1,7487012,https://binjolafilms.com/,http://www.phishtank.com/phish_detail.php?phis...,2022-04-12T16:26:54+00:00,yes,2022-04-12T16:42:46+00:00,yes,Other
2,7487000,https://aeonxq.tokyo/jp/login,http://www.phishtank.com/phish_detail.php?phis...,2022-04-12T16:08:18+00:00,yes,2022-04-12T16:22:56+00:00,yes,Other
3,7486984,https://lnkd.in/eZcmUWpr?ksfjjle,http://www.phishtank.com/phish_detail.php?phis...,2022-04-12T15:48:31+00:00,yes,2022-04-12T16:10:49+00:00,yes,Other
4,7486983,https://notificaciones.wpengine.com/cor/Correo...,http://www.phishtank.com/phish_detail.php?phis...,2022-04-12T15:47:51+00:00,yes,2022-04-12T16:10:49+00:00,yes,ING Direct


In [3]:
data0.shape

(9074, 8)

In [4]:
#Collecting 5,000 Phishing URLs randomly
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,7046111,https://3j124.csb.app,http://www.phishtank.com/phish_detail.php?phis...,2021-03-26T17:23:02+00:00,yes,2021-05-07T07:49:12+00:00,yes,Microsoft
1,7486353,https://folder909303-3u7878d78j893ik3.firebase...,http://www.phishtank.com/phish_detail.php?phis...,2022-04-12T08:37:42+00:00,yes,2022-04-12T08:42:46+00:00,yes,Microsoft
2,7421864,https://waqassupplies.com/admin/api.html,http://www.phishtank.com/phish_detail.php?phis...,2022-01-24T17:35:03+00:00,yes,2022-01-24T18:24:54+00:00,yes,Internal Revenue Service
3,7476607,https://knightfallfc.com/user1/linkedin/linkne...,http://www.phishtank.com/phish_detail.php?phis...,2022-04-01T04:56:42+00:00,yes,2022-04-01T05:03:02+00:00,yes,LinkedIn
4,7481001,https://www.mpdwe2fe.top/,http://www.phishtank.com/phish_detail.php?phis...,2022-04-06T17:14:45+00:00,yes,2022-04-06T17:22:27+00:00,yes,Other


In [5]:
phishurl.shape

(5000, 8)

In [6]:
#Loading legitimate files 
data1 = pd.read_csv("./data/Benign_list_big_final.csv")
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [7]:
#Collecting 5,000 Legitimate URLs randomly
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [8]:
legiurl.shape

(5000, 1)

In this step, features extraction from URL is done.

The extracted features are categorized into
1.   Address Bar based Features
2.   Domain based Features
3.   HTML & Javascript based Features
-----

#### Address Bar Based Features:
*   Domain of URL
*   IP Address in URL
*   @ Symbol in URL
*   Length of URL
*   Depth of URL
*   Redirection symbol // in URL
*   http/https in Domain name
*   Using URL Shortening Services usage
*   Prefix or Suffix - in Domain

In [9]:
# importing required packages for this section
from urllib.parse import urlparse
import ipaddress
import re

In [10]:
# 1.Domain of the URL
def getDomain(url):
  domain = urlparse(url).netloc
  if re.match(r"^www.", domain):
    domain = domain.replace("www.", "")
  return domain


In [11]:
# 2.Checks for IP address in URL
# If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip


In [12]:
# 3.Checks the presence of @ in URL
# Using @ symbol in the URL leads the browser to ignore everything preceding the @ symbol and the real address often follows the "@" symbol.
def haveAtSign(url):
  if "@" in url:
    at = 1     
  else:
    at = 0    
  return at

In [13]:
# 4.Finding the length of URL.
# Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

In [14]:
# 5.Gives number of '/' in URL
# Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'. The value of feature is a numerical based on the URL.
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [15]:
# 6.Checking for redirection '//' in the url
# The existence of "//" within the URL path means that the user will be redirected to another website. The location of the "//" in URL is computed. We find that if the URL starts with "HTTP", that means the "//" should appear in the sixth position. However, if the URL employs "HTTPS" then the "//" should appear in seventh position.
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

In [16]:
# 7.Existence of “HTTPS” Token in the Domain Part of the URL
# The phishers may add the "HTTPS" token to the domain part of a URL in order to trick users.
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

In [17]:
#listing shortening services
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [18]:
# 8. Checking for Shortening Services in URL
# If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

In [19]:
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain.
# Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1
    else:
        return 0

#### Domain Based Features:
*   DNS Record
*   Website Traffic 
*   Age of Domain
*   End Period of Domain


In [20]:
# 11.DNS Record availability
# For phishing websites, either the claimed identity is not recognized by the WHOIS database or no records founded for the hostname
# obtained in the main function of extaction itself

In [21]:
# 12.Web traffic
# This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as "Phishing".
# If the rank of the domain < 100000, the value of this feature is 1 (phishing) else 0 (legitimate).

def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 0
  else:
    return 1

In [22]:
# Another implementation of 12
def web_traffic_rank(url):
    alexa_base_url = 'https://alexa.com/siteinfo/'
    site_name = url
    site_name.lower()
    url_for_rank = alexa_base_url + site_name

    # Request formatted url for rank(s)
    page = requests.get(url_for_rank)
    soup = BeautifulSoup(page.content, 'html.parser')

    # select the data with class='rank-global' and the class='data'
    global_rank = soup.select('.rank-global .data')

    # Display Global rank safely
    rank = 0
    try:
        match = re.search(r'[\d,]+', global_rank[0].text.strip())
        rank = int(match.group())
    except:
        return 1
    if rank < 100000:
        return 0
    else:
        return 1

In [23]:
# 13.Survival time of domain: 
# # Returns the difference between termination time and creation time 
# Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 6 months for this project. Age here is nothing but different between creation and expiration time.
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) > 6):
      age = 0
    else:
      age = 1
  return age

In [24]:
# 14.End time of domain: 
# # The difference between termination time and current time 
# The end period considered for the legitimate domain is 6 months or less for this project.
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end

####  HTML and JavaScript based Features
*   IFrame Redirection
*   Status Bar Customization
*   Disabling Right Click
*   Website Forwarding

In [25]:
# 15. IFrame Redirection
# The end period considered for the legitimate domain is 6 months or less for this project.
# If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate)
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0
      else:
          return 1

In [26]:
# 16.Checks the effect of mouse over on status bar
# Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the "onMouseOver" event, and check if it makes any changes on the status bar. If response is empty although we didnt connect, it is 1.
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1
    else:
      return 0

In [27]:
# 17.Checks the status of the right click attribute
# Phishers may use JavaScript to show a fake URL in the status bar to users. To extract this feature, we must dig-out the webpage source code, particularly the "onMouseOver" event, and check if it makes any changes on the status bar
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [28]:
# 18.Checks the number of forwardings
# The fine line that distinguishes phishing websites from legitimate ones is how many times a website has been redirected. In our dataset, we find that legitimate websites have been redirected one time max. On the other hand, phishing websites containing this feature have been redirected at least 4 times
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

#### Now we have to call all the functions here for extraction of data.

In [30]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  
  #Domain based features (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  features.append(web_traffic(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))
  
  # HTML & Javascript based features (4)
  try:
    response = requests.get(url, timeout=5)
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)
  
  return features

In [31]:
legiurl.shape
URLS_COUNT=1000

In [ ]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0
for i in range(0, URLS_COUNT):
  url = legiurl['URLs'][i]
  print(f'{i+1} {url}')
  legi_features.append(featureExtraction(url,label))

In [95]:
# converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection',
                 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic',
                 'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over', 'Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns=feature_names)
legitimate.head()


,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,graphicriver.net,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0
1,ecnavi.jp,0,0,1,1,1,0,0,0,1,0,1,1,0,0,1,0,0
2,hubpages.com,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0
3,extratorrent.cc,0,0,1,3,0,0,0,0,1,1,1,1,0,0,1,0,0
4,icicibank.com,0,0,1,3,0,0,0,0,1,0,1,1,0,0,1,0,0


In [96]:
# Storing the extracted legitimate URLs fatures to csv file
legitimate.to_csv('./data/legitimate.csv', index= False)

In [97]:
phishurl.shape


(5000, 8)

In [98]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(0, URLS_COUNT):
  url = phishurl['url'][i]
  print(f'{i+1} {url}')
  phish_features.append(featureExtraction(url,label))

1 https://3j124.csb.app
2 https://folder909303-3u7878d78j893ik3.firebaseapp.com/
3 https://waqassupplies.com/admin/api.html
4 https://knightfallfc.com/user1/linkedin/linknewin/LINKEDIN/
5 https://www.mpdwe2fe.top/
6 https://www.hotforexxrd.cc/
7 http://addidentypagessecurity.co.vu/confirmid.php
8 https://esegui-procedura.com/
9 https://ershamshad.github.io/Login
10 https://www.maaaoacaseri.icu/
11 http://sn-28273739.ihostfull.com/?i=1
12 https://sistema.docssaude.com/LBIACCESINFORMATIONTECHACCOUNT/
13 https://sites.google.com/view/activationagrisecuriplus/accueil
14 https://pancakeswappshop.blogspot.com/
15 https://recoverphrase344.firebaseapp.com/
16 https://storage.cloud.google.com/dhngw6p6rwrwnuv6vnuse.appspot.com/index.html#brianvillacarlos@legalshieldcorp.com
17 https://www.uniswapfinancing.info/
18 https://sign-in-verification-929bb.web.app/#email=info@domain.tld
19 https://soumya252000.github.io/Netflix
20 http://grupchatviralhot24.barux2022.my.id
21 https://cancel629104-binance

In [99]:
# converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth', 'Redirection',
                 'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic',
                 'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over', 'Right_Click', 'Web_Forwards', 'Label']

phishing = pd.DataFrame(phish_features, columns=feature_names)
phishing.head()


,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,3j124.csb.app,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,1
1,folder909303-3u7878d78j893ik3.firebaseapp.com,0,0,1,0,0,0,0,1,1,1,1,1,0,0,1,0,1
2,waqassupplies.com,0,0,0,2,0,0,0,0,1,1,1,1,0,0,1,0,1
3,knightfallfc.com,0,0,1,4,0,0,0,0,1,1,1,1,0,0,1,0,1
4,mpdwe2fe.top,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,1


In [100]:
# Storing the extracted legitimate URLs fatures to csv file
phishing.to_csv('./data/phishing.csv', index= False)

#### Final Dataset generation after combining both

In the above section we formed two dataframes of legitimate & phishing URL features. Now, we will combine them to a single dataframe and export the data to csv file for the Machine Learning training done in other notebook. 

In [101]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)
urldata.head()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
0,graphicriver.net,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0
1,ecnavi.jp,0,0,1,1,1,0,0,0,1,0,1,1,0,0,1,0,0
2,hubpages.com,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0
3,extratorrent.cc,0,0,1,3,0,0,0,0,1,1,1,1,0,0,1,0,0
4,icicibank.com,0,0,1,3,0,0,0,0,1,0,1,1,0,0,1,0,0


In [102]:
urldata.tail()

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,DNS_Record,Web_Traffic,Domain_Age,Domain_End,iFrame,Mouse_Over,Right_Click,Web_Forwards,Label
1995,interbank-personas.web.app,0,0,0,0,0,0,0,1,1,1,1,1,0,0,1,0,1
1996,insurance2019.moneynet.com.tw,0,0,1,2,0,0,1,0,1,1,1,1,0,0,1,0,1
1997,jp-service-tranid-0001-0001n.b2ea63.cn,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1
1998,fedner.net,0,0,1,3,0,0,0,0,1,1,1,1,0,0,1,0,1
1999,kddio-fsi-com.etlzwfa.cn,0,0,0,0,0,0,0,1,1,1,1,1,1,0,1,0,1


In [103]:
urldata.shape

(2000, 18)

In [104]:
# Storing the data in CSV file
urldata.to_csv('./data/urldata.csv', index=False)